In [83]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor

# 한글폰트 설정, 그래프 마이너스 표시 설정

import matplotlib
from matplotlib import font_manager, rc
from matplotlib import pyplot as plt
import platform
import seaborn as sns

# 경로 설정

In [84]:
PATH = r'C:\Users\JY\JYC\Projects\parkingLot\data'

age_gender = pd.read_csv(PATH + '\\age_gender_info.csv')
train = pd.read_csv(PATH + '\\train.csv')
test = pd.read_csv(PATH + '\\test.csv')
submission = pd.read_csv(PATH + '\\sample_submission.csv')

# 폰트 설정

In [85]:
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

In [86]:
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17,A,9216000,82940,0.0,3,624,205
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17,A,12672000,107130,0.0,3,624,205
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17,A,12672000,107130,0.0,3,624,205
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17,A,18433000,149760,0.0,3,624,205
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17,A,18433000,149760,0.0,3,624,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,C2532,239,아파트,강원도,국민임대,49.20,19,7,A,11346000,116090,0.0,1,166,146
2865,C2532,239,아파트,강원도,국민임대,51.08,34,7,A,14005000,142310,0.0,1,166,146
2866,C2532,239,아파트,강원도,국민임대,51.73,34,7,A,14005000,142310,0.0,1,166,146
2867,C2532,239,아파트,강원도,국민임대,51.96,114,7,A,14005000,142310,0.0,1,166,146


In [87]:
age_gender

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,경기도,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,전라북도,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,...,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
5,강원도,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,...,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
6,광주광역시,0.031994,0.034566,0.049851,0.050254,0.065084,0.066875,0.066888,0.064416,0.080028,...,0.075930,0.059586,0.048552,0.031754,0.029749,0.010341,0.006343,0.000895,0.000353,0.000013
7,충청남도,0.031369,0.031711,0.059077,0.062422,0.067975,0.072622,0.065095,0.067303,0.078860,...,0.070278,0.057692,0.042296,0.028682,0.024514,0.010900,0.005429,0.001549,0.000219,0.000123
8,부산광역시,0.022003,0.022947,0.032681,0.035512,0.053796,0.057233,0.047049,0.048866,0.061952,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.002240,0.000268,0.000028
9,제주특별자치도,0.034690,0.036695,0.060094,0.063080,0.069135,0.069667,0.050808,0.048026,0.080548,...,0.074248,0.055717,0.047944,0.033054,0.026836,0.011332,0.006832,0.000982,0.000368,0.000082


# 컬럼명 변경

In [88]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

# age 고려

### 10대, 20~70대, 80대 이상으로 분류

In [89]:
under10 = []
under70 = []
under100 = []

In [90]:
female = []
male = []
for i in age_gender.columns:
    if '10대' in i:
        under10.append(i)
    elif '80대' in i:
        under100.append(i)
    elif '90대' in i:
        under100.append(i)
    elif '100대' in i:
        under100.append(i)
    else :
        under70.append(i)

In [91]:
under10_ratio = pd.DataFrame(age_gender[under10].sum(axis=1), columns = ['10대이하'])
under70_ratio = pd.DataFrame(age_gender[under70].sum(axis=1), columns = ['70대이하'])
under100_ratio = pd.DataFrame(age_gender[under100].sum(axis=1), columns = ['100대이하'])

In [92]:
place_index = pd.DataFrame(age_gender['지역'])
under10_ratio = pd.concat([under10_ratio, place_index], axis = 1)
under70_ratio = pd.concat([under70_ratio, place_index], axis = 1)
under100_ratio = pd.concat([under100_ratio, place_index], axis = 1)

under10_ratio = under10_ratio.set_index('지역')
under70_ratio = under70_ratio.set_index('지역')
under100_ratio = under100_ratio.set_index('지역')

In [93]:
train = pd.merge(train, under10_ratio, on='지역')
train = pd.merge(train, under70_ratio, on='지역')
train = pd.merge(train, under100_ratio, on='지역')

In [94]:
train.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
신분            0
임대보증금       569
임대료         569
지하철         207
버스            0
단지내주차면수       0
등록차량수         0
10대이하         0
70대이하         0
100대이하        0
dtype: int64

In [95]:
test.isna().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
신분            2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

# 지역명 숫자로 매핑

In [96]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i

In [97]:
train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

# 전용면적 5의 배수로 변경

In [98]:
train['전용면적'] = train['전용면적']//5*5
test['전용면적'] = test['전용면적']//5*5

# 전용면적 상/하한 적용

In [99]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [100]:
test['전용면적'].unique()

array([ 35.,  45.,  50.,  30.,  55.,  25.,  75., 100.,  15.,  20.,  40.,
        60.,  80.,  70.])

# 단지별 데이터 1차원으로 취합

In [111]:
columns = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스', '10대이하', '70대이하', '100대이하']
columns_test = ['단지코드', '총세대수', '공가수', '지역', '단지내주차면수', '지하철', '버스']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [112]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [113]:
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns_test:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()

411it [00:06, 59.56it/s]
150it [00:02, 62.74it/s]


In [114]:
new_train

,단지코드,총세대수,공가수,지역,단지내주차면수,지하철,버스,10대이하,70대이하,100대이하,...,면적_15.0,면적_100.0,면적_20.0,면적_40.0,면적_70.0,면적_80.0,면적_75.0,면적_60.0,면적_65.0,등록차량수
0,C2515,545.0,17.0,0.0,624.0,0.0,3.0,0.163127,0.792032,0.044841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,205.0
1,C2576,405.0,19.0,0.0,296.0,0.0,4.0,0.163127,0.792032,0.044841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0
2,C2416,560.0,0.0,0.0,154.0,0.0,5.0,0.163127,0.792032,0.044841,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0
3,C2013,595.0,3.0,0.0,509.0,0.0,3.0,0.163127,0.792032,0.044841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0
4,C1850,388.0,0.0,0.0,226.0,0.0,4.0,0.163127,0.792032,0.044841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,C1024,680.0,2.0,14.0,877.0,0.0,3.0,0.082280,0.831050,0.086669,...,0.0,0.0,0.0,0.0,216.0,111.0,0.0,0.0,0.0,957.0
407,C1652,890.0,20.0,14.0,423.0,1.0,8.0,0.082280,0.831050,0.086669,...,188.0,0.0,0.0,134.0,0.0,0.0,0.0,0.0,0.0,509.0
408,C2612,1684.0,27.0,15.0,1689.0,0.0,2.0,0.176979,0.797188,0.025833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1832.0
409,C1802,1438.0,3.0,15.0,1500.0,0.0,2.0,0.176979,0.797188,0.025833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1983.0


# 결측치 처리

In [115]:
new_train = new_train.fillna(-1)
new_test = new_test.fillna(-1)

# 학습

In [116]:
x_train = new_train.iloc[:, 1:-1]
y_train = new_train.iloc[:,-1]
x_test = new_test.iloc[:,1:]

In [117]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)

In [118]:
model.fit(x_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

# 추론 및 제출

In [119]:
pred = model.predict(x_test)

ValueError: X has 21 features, but DecisionTreeRegressor is expecting 24 features as input.

In [39]:
submission['num'] = pred

In [40]:
submission.to_csv('baseline.csv', index=False)